In [31]:
print("Hello")

Hello


In [7]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient
load_dotenv()

mongo_uri = os.getenv("MONGODB_URI")
client = MongoClient(mongo_uri)
collection = client["sample_mflix"]["ragpdf"]


In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Read the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

print("API key loaded:", openai_api_key is not None)


API key loaded: True


### RAG with MongoDB

In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [47]:
from openai import OpenAI

## Initialize the client
clientO = OpenAI()

## Specify the embedding model
model = "text-embedding-3-large"

## Defining the function to generate the embedding

def get_embedding(text, input_type = "document"):
    response = clientO.embeddings.create(
        model = model,
        input = text
    )
    return response.data[0].embedding

In [48]:
get_embedding("RAG Technology")

[-0.03790051490068436,
 -0.03420914337038994,
 -0.02823350764811039,
 -0.0022911173291504383,
 -0.041884273290634155,
 0.02127107046544552,
 -0.026406096294522285,
 0.02657056413590908,
 0.005628427490592003,
 0.042140111327171326,
 0.041811175644397736,
 -0.023537060245871544,
 0.0409705676138401,
 -0.031230462715029716,
 -0.02359188348054886,
 0.04371168464422226,
 -0.024925893172621727,
 -0.0106903575360775,
 0.0045936559326946735,
 -0.008154824376106262,
 0.02156345546245575,
 -0.018447719514369965,
 -0.01625482551753521,
 0.01868528313934803,
 -0.013632490299642086,
 -0.015560409054160118,
 -0.014125891029834747,
 0.005148732103407383,
 -0.06238783150911331,
 -0.002207741606980562,
 0.06169341504573822,
 0.03322234004735947,
 0.005404569674283266,
 -0.0007412438280880451,
 -0.015542135573923588,
 -0.01706802472472191,
 0.013522845692932606,
 0.029256857931613922,
 0.006263453047722578,
 -0.026972593739628792,
 0.02152690850198269,
 -0.015761423856019974,
 -0.035214219242334366,
 0

In [35]:
## Data Ingestion

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


## load the pdf

loader = PyPDFLoader("C:/Users/yashw/Downloads/Data Modelling --Proposal.pdf")
data = loader.load()

## Split the data into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 20)
documents = text_splitter.split_documents(data)




In [36]:
documents

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-03-17T15:38:06-05:00', 'author': 'Aasritha Manyam', 'moddate': '2025-03-17T15:38:06-05:00', 'source': 'C:/Users/yashw/Downloads/Data Modelling --Proposal.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Healthcare Equipment Inventory Management \nSystem \nProject Proposal  \n \nTeam Information \n• Team Member 1: Shiva Sai Sirigiri (11698776) \n• Team Member 2: Satish Reddy Tadi (11650147) \n• Team Member 3: Yashwanth Babu Gunisetty (11715340) \n• Team Member 4: Vishnu Priya Vulichi (11727751) \n• Team Member 5: Aasritha Sai Manyam (11708182) \nTopics Selected by Each Member'),
 Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-03-17T15:38:06-05:00', 'author': 'Aasritha Manyam', 'moddate': '2025-03-17T15:38:06-05:00', 'source': 'C:/Users/yashw

In [37]:
## prepare documents for insertion

docs_to_insert = [
    {
        "text": doc.page_content,
        "embedding": get_embedding(doc.page_content)
    } 
    
    for doc in documents
]

In [38]:
docs_to_insert

[{'text': 'Healthcare Equipment Inventory Management \nSystem \nProject Proposal  \n \nTeam Information \n• Team Member 1: Shiva Sai Sirigiri (11698776) \n• Team Member 2: Satish Reddy Tadi (11650147) \n• Team Member 3: Yashwanth Babu Gunisetty (11715340) \n• Team Member 4: Vishnu Priya Vulichi (11727751) \n• Team Member 5: Aasritha Sai Manyam (11708182) \nTopics Selected by Each Member',
  'embedding': [-0.03437415510416031,
   0.07070915400981903,
   -0.010955462232232094,
   0.00492438767105341,
   -0.0010036317398771644,
   -0.010948034934699535,
   -0.000620654784142971,
   0.016355205327272415,
   -0.006721826270222664,
   -0.010264711454510689,
   0.0014594903914257884,
   -0.06672804802656174,
   0.022193167358636856,
   -0.005696840584278107,
   -0.013049998320639133,
   0.04435662552714348,
   -0.03336402401328087,
   -0.006350454408675432,
   -0.01317626517266035,
   -0.02902640588581562,
   0.002892984775826335,
   0.017409902065992355,
   -0.001448349328711629,
   0.008615

In [ ]:
## Make the connection to the mongodb cluster to store the embeddings

from pymongo import MongoClient

client = MongoClient(os.getenv("MONGODB_URI"))
collection = client["sample_mflix"]["ragpdf"]

## Insert documents into the collection

result = collection.insert_many(docs_to_insert)
result



InsertManyResult([ObjectId('693289db6cfe433469f7d7ec'), ObjectId('693289db6cfe433469f7d7ed'), ObjectId('693289db6cfe433469f7d7ee'), ObjectId('693289db6cfe433469f7d7ef'), ObjectId('693289db6cfe433469f7d7f0'), ObjectId('693289db6cfe433469f7d7f1'), ObjectId('693289db6cfe433469f7d7f2'), ObjectId('693289db6cfe433469f7d7f3'), ObjectId('693289db6cfe433469f7d7f4'), ObjectId('693289db6cfe433469f7d7f5'), ObjectId('693289db6cfe433469f7d7f6'), ObjectId('693289db6cfe433469f7d7f7'), ObjectId('693289db6cfe433469f7d7f8'), ObjectId('693289db6cfe433469f7d7f9'), ObjectId('693289db6cfe433469f7d7fa'), ObjectId('693289db6cfe433469f7d7fb'), ObjectId('693289db6cfe433469f7d7fc'), ObjectId('693289db6cfe433469f7d7fd'), ObjectId('693289db6cfe433469f7d7fe'), ObjectId('693289db6cfe433469f7d7ff'), ObjectId('693289db6cfe433469f7d800'), ObjectId('693289db6cfe433469f7d801'), ObjectId('693289db6cfe433469f7d802'), ObjectId('693289db6cfe433469f7d803'), ObjectId('693289db6cfe433469f7d804'), ObjectId('693289db6cfe433469f7d8

In [40]:
## Query with search index

from pymongo.operations import SearchIndexModel
import time

## Creating Index Model, after this will create the Search Index
index_name = "vector_index"
search_index_model = SearchIndexModel(
    definition = {
        "fields": [
            {
                "type": "vector",
                "numDimensions": 3072,
                "path": "embedding",
                "similarity": "cosine"
            }
        ]
    },
    
    name = index_name,
    type = "vectorSearch"
)
collection.create_search_index(model=search_index_model)

'vector_index'

In [ ]:

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = mongo_uri

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [42]:
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")

predicate = None

if predicate is None:
    predicate = lambda index: index.get("queryable") is True

while True:
    indices = list(collection.list_search_indexes(index_name))

    if len(indices) and predicate(indices[0]):
        break

    time.sleep(5)

print(index_name + " is ready for querying.")


Polling to check if the index is ready. This may take up to a minute.
vector_index is ready for querying.


In [43]:
## Defining a function to run vector search queries

def get_query_results(query):
    
    query_embedding = get_embedding(query, input_type="query")
    print(query_embedding)
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 3072,
                "limit": 5
            }
        }, {
            "$project": {
                "_id": 0,
                "text": 1
            }
        }
    ]
    
    results = collection.aggregate(pipeline)
    print(results)
    
    array_of_results = []
    for doc in results:
        array_of_results.append(doc)
    return array_of_results

In [49]:
get_query_results("User Requirements")

[-0.0074173882603645325, -0.0027117333374917507, -0.0001383283088216558, 0.021205943077802658, 0.025841224938631058, 0.02122470922768116, -0.016373615711927414, 0.05922650918364525, -0.014562665484845638, 0.01987353339791298, 0.06245431676506996, -0.010884466581046581, 0.0019997861236333847, -0.018597424030303955, -0.005841017235070467, 0.016073353588581085, -0.023232705891132355, -0.019029049202799797, -0.02000489830970764, -0.004072291776537895, 0.038377124816179276, 0.02668570913374424, -0.00486516859382391, 0.0041497028432786465, 0.004428851883858442, 0.006525988224893808, -0.008862394839525223, -0.025634795427322388, -0.01987353339791298, -0.0005700269830413163, -0.002236710861325264, 0.018888302147388458, 0.021037045866250992, 0.0010802363976836205, 0.0022285005543380976, -0.027567725628614426, 0.031883981078863144, -0.0055829803459346294, -0.04008486121892929, 0.036219000816345215, 0.0013629040913656354, 0.002167510101571679, 0.00234344438649714, 0.04368799552321434, 0.028975199

[{'text': '- Analysis of cost and planning of budget for equipment purchases \n \nUser Requirements \n \nThe system must have a comprehensive register of all medical equipment with their model, and \nserial number, date of acquisition, expected lifespan, and current location. \nInventory Checking: The system needs to store information for all medical equipment such as the'},
 {'text': '- Analysis of cost and planning of budget for equipment purchases \n \nUser Requirements \n \nThe system must have a comprehensive register of all medical equipment with their model, and \nserial number, date of acquisition, expected lifespan, and current location. \nInventory Checking: The system needs to store information for all medical equipment such as the'},
 {'text': 'Administrators will have relative privileges. Technicians will update maintenance records and \nManagers will access reports. \nThe system will have APIs to allow connection with other existing systems within the healthcare \norganiz

In [52]:
from openai import OpenAI

## Specify the query, retrieve the relevant doc and converting it to string

query = "Give me the topics selected by each team member from project Healthcare equipment inventory management system"
context_docs = get_query_results(query)
context_string = " ".join([doc["text"] for doc in context_docs])

## Constructing prompt for the LLM to give the output in a manner, using retrieved documents as source

prompt = f"""use the following pieces of context to answer the question at the end.
    {context_string}
    Question: {query}
"""   
openai_client = OpenAI()


#Open AI model to use for giving the response

model_name = "gpt-5.1"

completion = openai_client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}]
)

print(completion.choices[0].message.content)

[-0.02364811860024929, 0.036710888147354126, -0.001741434563882649, -0.0004240473499521613, -0.0014709692914038897, 0.0037362417206168175, 0.00887206569314003, 0.001351320999674499, 0.012716895900666714, 0.006567581091076136, 0.028313392773270607, -0.0668582171201706, 0.04092572256922722, 0.01989981159567833, 0.005220281891524792, 0.008373363874852657, -0.025691187009215355, 0.011269052512943745, -0.002149646170437336, -0.04298487678170204, 0.018435880541801453, 0.016272157430648804, -0.0030404722783714533, 0.020205466076731682, -0.0030847119633108377, -0.006635951343923807, 0.011840146034955978, 0.002284375950694084, -0.025224659591913223, -0.0055943080224096775, 0.009274245239794254, 0.03077472746372223, -0.008638801984488964, -0.03436216339468956, -0.01895066909492016, -0.01798543892800808, 0.004134398885071278, 0.031048210337758064, 0.013304077088832855, 0.0356491357088089, 0.021621135994791985, -0.011775797232985497, -0.01862892508506775, 0.045977089554071426, -0.02969688922166824